In [5]:
	def parse(self, response):
		for num in [1,2]:
            rows = response.xpath('//*[@id="template-home-v2"]/div[1]/div[4]/div/div[2]/div[1]/div[%d]'%num) #for row in rows, [1] to [9] #this gets the first row
            pattern = './a[2]/@title' #or a/@title
            for row in rows:
                title = row.xpath(pattern).extract_first()
                print (title)

		item = NycdsaItem()
		item['title'] = title	
		yield item 






1
2


In [3]:
import scrapy

class WikiItem(scrapy.Item):
    film = scrapy.Field()
    year = scrapy.Field()
    awards = scrapy.Field()
    nominations = scrapy.Field()
#__init__.py is for packages

In [7]:
#the wiki_spider.py document
#import scrapy.Spider (from which our spider will inherit), 
#and the item class we defined in items.py
from scrapy import Spider
from wiki.wiki.items import WikiItem

# define our spider class, and give it its 3 must have/ required attributes
class WikiSpider(Spider):
    name = 'wiki_spider'
    allowed_urls = ['https://en.wikipedia.org']
    start_urls = ['https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films']
    
    def parse(self, response):
        # Find all the table rows
        rows = response.xpath('//*[@id="mw-content-text"]/div/table/tbody/tr')

        # The movie title could be of different styles so we need to provide all the possibilities.
        patterns = ['./td[1]/i/a/text()', './td[1]/i/b/a/text()', './td[1]/i/span[2]//text()', './td[1]/i/b/span/text()']
        for row in rows:
            # extract() will return a Python list, extract_first() will return the first element in the list
            # If you know the first element is what you want, you can use extract_first()
            for pattern in patterns:
            film = row.xpath(pattern).extract_first()
                if film:
                    break
            # If the movie title is missing, then we just skip it.
            if not film: # non-type object 
                continue # back to the for loop for rows in rows 
            # Relative xpath for all the other columns
            year = row.xpath('./td[2]/a/text()').extract_first()
            awards = row.xpath('./td[3]/text()').extract_first()
            nominations = row.xpath('./td[4]/text()').extract_first().strip()

            # Initialize a new WikiItem instance for each movie.
            item = WikiItem()
            item['film'] = film
            item['year'] = year
            item['awards'] = awards
            item['nominations'] = nominations
            yield item

# after yielding item from spider, next is pipeline, below

In [ ]:
#https://doc.scrapy.org/en/latest/topics/selectors.html

In [ ]:
# //div[@class="mine"] selects all the <div> tags having an attribute class="mine"
# class is an attribute

#https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films
<table class ="wikitable sortable jquery-tablesorter"> 
#right click, copy, XPath :
/html/body/div[3]/div[3]/div[4]/div/table #firefox
//*[@id="mw-content-text"]/div/table #chrome Xpath for the entire table
# / tbody /tr #append tbody and tr to Xpath

##########################################
<thead>...</thead>
<tbody>

<tr>...</tr> #each tr is a different movie
<tr>...</tr>
<tr>...</tr>
<tr>...</tr>
<tr>...</tr>

</tbody>
<tfoot></tfoot>
</table>

In [ ]:
######## START SCRAPY SHELL IN ANACONDA IN PY3 ####################

In [ ]:
# right click anaconda, start second terminal
# cd into project folder

cd C:\Users\asus\NYC Data Science Academy\NYCDSA Unit 5 Data Analysis with Python\Project 2 - Web Scraping

scrapy shell "https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films"
           # "url_of_the_page_you're_scraping"

rows = response.xpath('//*[@id="mw-content-text"]/div/table/tbody/tr') #rows, points to the entire table
row = rows[3] #define row

In [9]: print (row)
<Selector xpath='//*[@id="mw-content-text"]/div/table/tbody/tr' data='<tr>\n<td><i><a href="/wiki/Three_Billboa'>

In [18]: xtext = row.xpath('./td[1]/i/a/text()') #define xtext through row.xpath

In [19]: xtext
Out[19]: [<Selector xpath='./td[1]/i/a/text()' data='Three Billboards Outside Ebbing, Missour'>]
#In [20]: print (xtext)
#[<Selector xpath='./td[1]/i/a/text()' data='Three Billboards Outside Ebbing, Missour'>]

In [21]: xtext.extract() #use .extract() to print value
Out[21]: ['Three Billboards Outside Ebbing, Missouri']
    #extract_first() returns none if extracting from empty list. 
    
film = row.xpath('./td[1]/i/a/text()').extract_first()

In [ ]:
#pipelines.py
# -*- coding: utf-8 -*-

# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://doc.scrapy.org/en/latest/topics/item-pipeline.html


from scrapy.exceptions import DropItem
from scrapy.exporters import CsvItemExporter

class ValidateItemPipeline(object):

    def process_item(self, item, spider):
        if not all(item.values()):
            raise DropItem("Missing values!")
        else:
            return item

class WriteItemPipeline(object):

    def __init__(self):
        self.filename = 'academy_awards.csv'

    def open_spider(self, spider):
        self.csvfile = open(self.filename, 'wb')
        self.exporter = CsvItemExporter(self.csvfile)
        self.exporter.start_exporting()

    def close_spider(self, spider):
        self.exporter.finish_exporting()
        self.csvfile.close()

    def process_item(self, item, spider):
        self.exporter.export_item(item)
        return item
#next edit settings.py
#then, ready to deploy spider

In [ ]:
#settings.py
# -*- coding: utf-8 -*-

# Scrapy settings for wiki project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://doc.scrapy.org/en/latest/topics/settings.html
#     https://doc.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://doc.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'wiki'

SPIDER_MODULES = ['wiki.spiders']
NEWSPIDER_MODULE = 'wiki.spiders'

ITEM_PIPELINES = {'wiki.pipelines.ValidateItemPipeline': 100, #smaller number -> higher priority
					'wiki.pipelines.WriteItemPipeline': 200}
#validate first, then write item to file

# Crawl responsibly by identifying yourself (and your website) on the user-agent
#USER_AGENT = 'wiki (+http://www.yourdomain.com)'

# Obey robots.txt rules
ROBOTSTXT_OBEY = True

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32

# Configure a delay for requests for the same website (default: 0)
# See https://doc.scrapy.org/en/latest/topics/settings.html#download-delay
# See also autothrottle settings and docs
#DOWNLOAD_DELAY = 3
# The download delay setting will honor only one of:
#CONCURRENT_REQUESTS_PER_DOMAIN = 16
#CONCURRENT_REQUESTS_PER_IP = 16

# Disable cookies (enabled by default)
#COOKIES_ENABLED = False

# Disable Telnet Console (enabled by default)
#TELNETCONSOLE_ENABLED = False

# Override the default request headers:
#DEFAULT_REQUEST_HEADERS = {
#   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#   'Accept-Language': 'en',
#}

# Enable or disable spider middlewares
# See https://doc.scrapy.org/en/latest/topics/spider-middleware.html
#SPIDER_MIDDLEWARES = {
#    'wiki.middlewares.WikiSpiderMiddleware': 543,
#}

# Enable or disable downloader middlewares
# See https://doc.scrapy.org/en/latest/topics/downloader-middleware.html
#DOWNLOADER_MIDDLEWARES = {
#    'wiki.middlewares.WikiDownloaderMiddleware': 543,
#}

# Enable or disable extensions
# See https://doc.scrapy.org/en/latest/topics/extensions.html
#EXTENSIONS = {
#    'scrapy.extensions.telnet.TelnetConsole': None,
#}

# Configure item pipelines
# See https://doc.scrapy.org/en/latest/topics/item-pipeline.html
#ITEM_PIPELINES = {
#    'wiki.pipelines.WikiPipeline': 300,
#}

# Enable and configure the AutoThrottle extension (disabled by default)
# See https://doc.scrapy.org/en/latest/topics/autothrottle.html
#AUTOTHROTTLE_ENABLED = True
# The initial download delay
#AUTOTHROTTLE_START_DELAY = 5
# The maximum download delay to be set in case of high latencies
#AUTOTHROTTLE_MAX_DELAY = 60
# The average number of requests Scrapy should be sending in parallel to
# each remote server
#AUTOTHROTTLE_TARGET_CONCURRENCY = 1.0
# Enable showing throttling stats for every response received:
#AUTOTHROTTLE_DEBUG = False

# Enable and configure HTTP caching (disabled by default)
# See https://doc.scrapy.org/en/latest/topics/downloader-middleware.html#httpcache-middleware-settings
#HTTPCACHE_ENABLED = True
#HTTPCACHE_EXPIRATION_SECS = 0
#HTTPCACHE_DIR = 'httpcache'
#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'

In [ ]:
#deploy spider
#open new anaconda prompt, activate py3, cd to folder with scrapy.cfg
# (ipykernel_py3) C:\Users\asus\NYC Data Science Academy\
# NYCDSA Unit 5 Data Analysis with Python\Project 2 - Web Scraping\wiki
scrapy crawl wiki_spider

In [ ]:
############# NEW PROJECT ##################

In [ ]:
scrapy startproject budget #create new scrapy project
#can drag folder into file editor to see structure

#create files
#update items.py
import scrapy
class BudgetItem(scrapy.Item):
    # define the fields for your item here like:
    RDate = scrapy.Field()
    Title = scrapy.Field()
    PBudget = scrapy.Field()
    DomesticG = scrapy.Field()
    WorldwideG = scrapy.Field()
    
#create budget_spider.py under budget/spiders/
#Definitely the first thing to do is to import 
#the module and the budget item we defined in items.py
from scrapy import Spider
from budget.items import BudgetItem

#define spider called BudgetSpider, give it a name called budget_spider
#use budget_spider to run a crawl on terminal

class BudgetSpider(Spider):
    name = "budget_spider"
    allowed_urls = ['https://www.the-numbers.com/']
    start_urls = ['https://www.the-numbers.com/movie/budgets/all']
    def parse(self, response):
            pass



In [ ]:
#ctrl+d to quit scrapy shell
#start new scrapy shell in py3 env
scrapy shell "https://www.the-numbers.com/movie/budgets/all"
#403 forbidden
#add header to request for 200 
scrapy shell -s USER_AGENT="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36" "https://www.the-numbers.com/movie/budgets/all"
#success
#In [1]: response
#Out[1]: <200 https://www.the-numbers.com/movie/budgets/all>

rows = response.xpath('//*[@id="page_filling_chart"]/center/table/tbody/tr')
#doesnt work, use:
rows = response.xpath('//*[@id="page_filling_chart"]/center/table//tr') #tbody not in code, added in by google chrome
#view page source to see actual code

#Release Date:
In [11]: rows[1].xpath('./td[2]/a/text()').extract_first()
Out[11]: '12/18/2009'

#Movie Title:
In [12]: rows[1].xpath('./td[3]/b/a/text()').extract_first()
Out[12]: 'Avatar'

#3 more rows, PBudget, DomesticG, WorldwideG


In [ ]:
# PAGE 45

In [ ]:
#ctrl+d to quit scrapy shell

In [ ]:
#cls to clear screen #ctrl+L to clear screen in scrapy shell